![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Agentic tasks with Agent Framework and Redis

This notebook demonstrates how to build an agent using Microsoft's [Agent Framework](https://github.com/microsoft/agent-framework/tree/main).

It adapts the work found in [Redis-AI-Resources Autogen Agent Tutorial](https://github.com/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/04_autogen_agent.ipynb), and simply applies the setting in Agent Framework instead.

We'll define an agent, give it access to tools and memory, then set in on a task to see how it uses its abilities.

## Note
Agent Framework relies on either the OpenAI or AzureOpenAI API for LLM capability. This notebook will rely on the OpenAI API.

## Let's Begin!

<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/05_agent_framework_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
# %pip install agent-framework --pre
# %pip install -q "redisvl>=0.8.0" sentence-transformers openai tiktoken python-dotenv redis google pandas

## For Colab download and run a Redis instance




In [1]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


## Format plain text into Markdown-style blockquotes for display in a Colab notebook

In [2]:
import textwrap

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Import Dependencies

In [3]:
import asyncio
import os

import json
import os
import re
import requests
from collections import Counter
from typing import List

In [4]:
# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


## Building our agent

We'll be building a restaurant review writing agent that takes in a set of restaurant reviews, collects relevant information, and provides a summary and analysis you can use for SEO.

### Defining tools
One of the defining the features of an agent is its ability to use tools so let's give it some. Our agent will decide when to call each tool, construct the appropriate arguments, then retrieve and utilize the results.

With Autogen that just requires we define a well named function with type hints in its signature.

We will have three main tools:
1. A `summarize()` function that can take in a collection of reviews and boil them all down to a single summary.
2. A `get_keywords()` function to count the most common words present in the article, becuase LLM's often struggle with character counting.
3. A `publish_article()` function that will write our final article to a separate file we can then upload elsewhere.

In [5]:

async def summarize(restaurant_name: str, all_reviews: List[str]) -> str:
    """takes a list of reviews for a single restaurant and returns a summary of all of them."""
    # set up a summarizer model
    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
    # pass all the reviews
    summary = summarizer('\n'.join(all_reviews),  # concatenate all the reviews together
                max_length=1024,
                min_length=128,
                do_sample=False)[0]["summary_text"]
    return restaurant_name + ": " + summary


async def get_keywords(full_text: str) -> List[str]:
    """extract the most commonly occurring keywords present in the reviews to know
    which terms it is likely to rank highly for in keyword search engines."""
    # define a set of common English stopwords to ignore
    STOPWORDS = {
        'the', 'of', 'and', 'to', 'for', 'in', 'on', 'at', 'a', 'an', 'is', 'it', 'its', 'with', 'as', 'by', 'from', 'that',
        'this', 'those', 'be', 'are', 'was', 'were', 'or', 'but', 'not', 'so', 'if', 'then', 'than', 'which', 'who', 'whom',
        'about', 'into', 'out', 'up', 'down', 'over', 'under', 'again', 'further', 'once', 'here', 'there', 'when',
        'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
        'nor', 'only', 'own', 'same', 'can', 'will', 'just', 'don', 'should', 'now', 'has', 'have', 'had', 'do', 'does',
        'did', 'their', 'them', 'they', 'you', 'your', 'yours', 'he', 'him', 'his', 'she', 'her', 'hers', 'we', 'us',
        'our', 'ours', 'i', 's', 'me', 'my', 'mine', 'also', 'place'
    }
    # remove punctuation and lowercase the text
    words = re.findall(r'\b\w+\b', full_text.lower())
    # filter out stopwords
    filtered_words = [word for word in words if word not in STOPWORDS]
    # count occurrences
    word_counts = Counter(filtered_words)
    # return the top 10
    return [word for word, _ in word_counts.most_common(10)]


async def publish_article(final_draft: str, file_name:str= "food_article.md") -> str:
    "accepts the final version of an article, writes it to a markdown file and returns the full file location path."
    with open(file_name, 'w') as file:
        file.write(final_draft)

    full_path = os.path.abspath(__file__)
    return os.path.join(full_path, file_name)

## Adding relevant memories
Our agent needs to know what people think of these restaurants so we'll add the user reviews to our agent memory powered by Redis.


In [6]:
# fetch the reviews from our public S3 bucket
# the original dataset can be found here: https://www.kaggle.com/datasets/jkgatt/restaurant-data-with-100-trip-advisor-reviews-each
def fetch_data(file_name):
    dataset_path = 'datasets/'
    try:
        with open(dataset_path + file_name, 'r') as f:
            return json.load(f)
    except:
        url = 'https://redis-ai-resources.s3.us-east-2.amazonaws.com/recommenders/datasets/two-towers/'
        r = requests.get(url + file_name)
        if not os.path.exists(dataset_path):
            os.makedirs(dataset_path)
        with open(dataset_path + file_name, 'wb') as f:
            f.write(r.content)
        return json.loads(r.content.decode('utf-8'))

In [7]:
restaurant_data = fetch_data('factual_tripadvisor_restaurant_data_all_100_reviews.json')

print(f"we have {restaurant_data['restaurant_count']} restaurants in our dataset, with {restaurant_data['total_review_count']} total reviews")

restaurant_reviews = restaurant_data["restaurants"] # ignore the count fields

# drop some of the fields that we don't need
for restaurant in restaurant_reviews:
    for field in ['region', 'country', 'tel', 'fax', 'email', 'website', 'address_extended', 'chain_name', 'trip_advisor_url']:
        if field in restaurant:
            restaurant.pop(field)

we have 147 restaurants in our dataset, with 14700 total reviews


## Initialize Redis Provider + Vectorizer

In [ ]:
from agent_framework import ChatMessage, Role
from agent_framework.openai import OpenAIChatClient
from agent_framework_redis._chat_message_store import RedisChatMessageStore
from agent_framework_redis._provider import RedisProvider
from redisvl.extensions.cache.embeddings import EmbeddingsCache
from redisvl.utils.vectorize import OpenAITextVectorizer
from logging import WARNING, getLogger
from tqdm import tqdm

logger = getLogger()
logger.setLevel(WARNING)

REDIS_OPENAI_EMBEDDINGS_CACHE_NAME = os.getenv("REDIS_OPENAI_EMBEDDINGS_CACHE_NAME",
                                               "openai_embeddings_cache")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY",
                         "<YOUR OPENAI API KEY>")
OPENAI_CHAT_MODEL_ID=os.getenv("OPENAI_CHAT_MODEL_ID",
                               "gpt-5-mini")
OPENAI_EMBEDDING_MODEL_ID=os.getenv("OPENAI_EMBEDDING_MODEL_ID",
                                    "text-embedding-3-small")

thread_id="test_thread"

vectorizer = OpenAITextVectorizer(
    model=OPENAI_EMBEDDING_MODEL_ID,
    api_config={"api_key": OPENAI_API_KEY},
    cache=EmbeddingsCache(name=REDIS_OPENAI_EMBEDDINGS_CACHE_NAME,
                          redis_url=REDIS_URL)
)

provider = RedisProvider(
    redis_url=REDIS_URL,
    index_name="restaurant_reviews",
    prefix="trip_advisor",
    application_id="restaurant_reviews_app",
    agent_id="restaurant_reviews_agent",
    user_id="restaurant_reviews_user",
    redis_vectorizer=vectorizer,
    vector_field_name="vector",
    vector_algorithm="hnsw",
    vector_distance_metric="cosine",
    thread_id=thread_id,
    overwrite_index=True
)


### Add Memories in batches to the Redis Memory provider

In [ ]:
batch_size=128
messages=[]
await provider.thread_created(thread_id=thread_id)
for restaurant in tqdm(restaurant_reviews):
    # add each review to our agent memory
    # for brevity we'll take only the first 10 reviews per restaurant
    for review in restaurant['reviews'][:10]:
        try:
          review_title=review['review_title']
          review_text=review["review_text"]
          meta_data=str("\n".join([str(key) + ": " + str(val) for key, val in restaurant.items() if key != "reviews"]))
          memory="\n".join([review_title, review_text, meta_data])
          message = ChatMessage(role='system', conversation_id=thread_id, text=memory)
          messages.append(message)
          if len(messages)==batch_size:
            await provider.invoked(request_messages=[message])
            messages=[]
        except Exception as e:
          print(e)

In [19]:
all=await provider.search_all()
# Number of Reviews
print(len(all))

# Sample Review
print(all[100]["content"])


1118
This place is the best!
First let me start by saying I am a local who eats here at least once a week if not twice.I travel for a living and try a lot of different restaurants and Nani's is by far the bestCuban food I have ever eaten. Everything is fresh &amp; just so full of flavor! My favorites are the Empanadas, Cuban sandwich, Crab Cakes and of course the black beans. When you go there, ask for Marie's homemade hot sauce &amp; Mojo. Ray &amp; Marie are the nicest &amp; really make you feel at home.
name: Nanis Coffee
address: 2739 Geary Blvd
locality: San Francisco
latitude: 37.782187
longitude: -122.448613
cuisine: ['Coffee', 'Sandwiches', 'Cafe', 'Bagels', 'Tea']
price: 1
rating: 5.0
hours: {'monday': [['7:00', '17:30']], 'tuesday': [['7:00', '17:30']], 'wednesday': [['7:00', '17:30']], 'thursday': [['7:00', '17:30']], 'friday': [['7:00', '17:30']], 'saturday': [['8:30', '17:00']], 'sunday': [['8:30', '15:00']]}
parking: True
parking_valet: False
parking_garage: False
parking

### Initialize Agent with Provider and Chat Message Store

In [22]:
chat_message_store_factory = lambda: RedisChatMessageStore(
    redis_url=REDIS_URL,
    thread_id="test_thread",
    key_prefix="chat_messages",
    max_messages=100
)

client = OpenAIChatClient(model_id=OPENAI_CHAT_MODEL_ID,
                          api_key=OPENAI_API_KEY)
# Create agent wired to the Redis context provider. The provider automatically
# persists conversational details and surfaces relevant context on each turn.
review_agent = client.create_agent(
    name="MemoryEnhancedAssistant",
    instructions=(
        "You are a helpful assistant. Personalize replies using provided context. "
        "Before answering, always check for stored context"
    ),
    tools=[summarize , get_keywords, publish_article],
    context_providers=provider,
    chat_message_store_factory=chat_message_store_factory,
)

### Teaching Preferences

In [24]:
preference = "Remember that I hate beef"
result = await review_agent.run(preference)
print(result)

Got it — I remember you hate beef. I’ll avoid recommending beef dishes and beef‑focused restaurants going forward.

Would you like a beef‑free shortlist now? (Options: by neighborhood, by price tier, by cuisine, or for a special occasion.)


### Running a Streaming Task

In [23]:

import asyncio

async def streaming_example(agent, query) -> None:
      print(f"User: {query}")
      print("Agent: ", end="", flush=True)
      async for chunk in agent.run_stream(query):
          if chunk.text:
              print(chunk.text, end="", flush=True)
      print("\n")

writing_task = "Write an article reviewing the restaurants in the San Francisco bay area. \
            Include a brief summary of the most popular restaurants based on the user reviews. \
            Group them into categories based on their cuisine and price, and talk about the \
            top rated restaurants in each category."

await streaming_example(review_agent, writing_task)

User: Write an article reviewing the restaurants in the San Francisco bay area.             Include a brief summary of the most popular restaurants based on the user reviews.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.
Agent: Thanks — I remembered you hate beef, so this review highlights non‑beef options and avoids pushing steak/BBQ spots. Below is an article-style overview of the San Francisco Bay Area restaurant scene, a short summary of the most popular places based on user reviews (including a few from your saved visits), and a cuisine × price breakdown with the top-rated picks and what reviewers typically say about them.

San Francisco Bay Area — a quick dining profile
The Bay Area mixes landmark institutions with inventive newcomers and excellent ethnic restaurants. Reviewers reward seasonality and ingredient quality at fine dining spots, authenticity at ethnic counters and taqueri

That's a lot of output from our agent, and it can be hard to parse through all of it.

There's no need for us to read it closely as we'll have the final article written to an external file cleanly when we're - or rather our agent - is finished.

## Follow up tasks
Our agent doesn't have to be a one-and-done worker. We can ask it to continue toward our overall goal of having a well viewed food critic article.

You've probably noticed the agent's output is somewhat messy. That's ok as our final article will be written cleanly to a markdown file.

In [27]:
task_list = ["Now analyze your article and tell me the key search terms it is likely to rank highly for.",
             "Using your analysis suggest changes to the original article to improve keyword ranking.",
             "Based on your suggestions, edit and modify your article to improve SEO keyword ranking. Give a new list of top keywords",
             "When it is ready, publish the article by saving it to a markdown file."]

for task in task_list:
  await streaming_example(review_agent, task)

User: Now analyze your article and tell me the key search terms it is likely to rank highly for.
Agent: I ran a keyword analysis on the article and refined the raw results. Below are the key search terms the article is likely to rank well for, grouped by priority and intent, plus a few quick SEO suggestions to improve ranking.

Primary (high relevancy / local intent)
- San Francisco Bay Area restaurants
- San Francisco restaurants
- best restaurants San Francisco
- SF restaurants by cuisine
Why: article repeatedly uses these exact phrases, mentions neighborhoods and many SF restaurant names.

Secondary (strong, content-backed niche terms)
- best seafood San Francisco
- San Francisco vegetarian restaurants / plant-forward San Francisco
- San Francisco tasting menu / SF tasting menu
- best omakase San Francisco
- best dim sum San Francisco
- Ferry Building restaurants San Francisco
Why: article emphasizes seafood, vegetarian/vegan, tasting menus, omakase and dim sum and references Ferry 

## The finished product
We got another large block of agent output showing us it's hard work. What we really care about it is the finished product. Check your local directory for a markdown file with our finished article.

That's it!